In [122]:
import pandas as pd
import numpy as np 

import plotly.graph_objs as go

from website.utils.graphs import add_custom_legend_axis
from website import MAIN_CATEGORIES, CATEGORIES, ALGORITHMS, SCORES, FOLDS


scores_data = pd.read_feather(f"../data/scores.feather").to_dict()
targets = ["all"]
examination_categories, laboratory_categories, questionnaire_categories = ["Body__Measures"], ["all"], ["all"]
algorithms = ["elastic_net", "light_gbm"]
metric = "diff_c_index"


scores_full = pd.DataFrame(scores_data).set_index(["main_category", "category"])
scores_full.columns = pd.MultiIndex.from_tuples(
    list(map(eval, scores_full.columns.tolist())), names=["target", "algorithm", "fold", "metric"]
)

categories_to_display = {
    "examination": examination_categories,
    "laboratory": laboratory_categories,
    "questionnaire": questionnaire_categories,
}

indexes_to_take = []
indexes_to_rename = {}

for main_category in MAIN_CATEGORIES:
    indexes_to_rename[main_category] = MAIN_CATEGORIES[main_category]
    if categories_to_display[main_category] == ["all"]:
        categories_to_display[main_category] = (
            pd.Index(list(CATEGORIES[main_category].keys())).drop("all").to_list()
        )
    for category in categories_to_display[main_category]:
        indexes_to_rename[category] = CATEGORIES[main_category][category]
        indexes_to_take.append([main_category, category])
for algorithm in algorithms:
    indexes_to_rename[algorithm] = ALGORITHMS[algorithm]

scores = scores_full.loc[indexes_to_take, (targets, ["numbers", "age_ranges"] + algorithms)]
scores.rename(index=indexes_to_rename, inplace=True)

if targets != ["age"]:
    scores.replace(-1, np.nan, inplace=True)

In [123]:
hovertemplate = (
    "%{x},<Br> score: %{y:.3f} +- %{customdata[0]:.3f}, %{customdata[1]} participants with %{customdata[2]} variables, age range %{customdata[3]} to %{customdata[4]} years old <extra>%{customdata[5]}</extra>"
)

figures = {}
for fold in FOLDS:
    scores_fold = scores.loc[:, (slice(None), slice(None), ["numbers", "age_ranges", fold])].droplevel("fold", axis="columns")

    x_positions = pd.Series(np.arange(5, 10 * len(scores_fold.index) + 5, 10), index=scores_fold.index)

    figures[fold] = go.Figure()
    figures[fold].update_layout(
        xaxis={
            "tickvals": np.arange(5, 10 * len(scores_fold.index) + 5, 10),
            "ticktext": [" - ".join(elem) for elem in scores_fold.index],
        }
    )
    
    for target in targets:
        for algorithm in algorithms:
            customdata = np.dstack(
                (
                    scores_fold[(target, algorithm, f"{metric}_std")].values.flatten(),
                    scores_fold[(target, "numbers", "n_participants")].values.flatten(),
                    scores_fold[(target, "numbers", "n_variables")].values.flatten(),
                    scores_fold[(target, "age_ranges", "min")].values.flatten().astype(int),
                    scores_fold[(target, "age_ranges", "max")].values.flatten().astype(int),
                    [ALGORITHMS[algorithm]] * len(scores_fold.index),
                )
            )[0]

            figures[fold].add_bar(
                x=x_positions.loc[scores_fold.index].values.flatten(),
                y=scores_fold[(target, algorithm, metric)],
                error_y={"array": scores_fold[(target, algorithm, f"{metric}_std")], "type": "data"},
                name=ALGORITHMS[algorithm],
                hovertemplate=hovertemplate,
                customdata=customdata,
            ) 

    add_custom_legend_axis(figures[fold], scores_fold.index, -120 if metric == "rmse" else -1, -60 if metric == "rmse" else -0.5, min(scores_fold.loc[:, (targets, algorithms, metric)].min().min(), 0))
    
    figures[fold].update_layout(
        yaxis={
            "title": SCORES[targets[0]][metric],
            "showgrid": False,
            "zeroline": False,
            "showticklabels": True,
            "title_font": {"size": 45},
            "dtick": 12 if metric == "rmse" else 0.1,
            "tickfont_size": 20,
        },
        xaxis={"showgrid": False, "zeroline": False},
        height=800,
        margin={"l": 0, "r": 0, "b": 0, "t": 0},
        legend={"orientation": "h", "yanchor": "bottom", "font": {"size": 30}},
    )
    break

figures["train"].show()


In [21]:
scores
hovertemplate = (
    "%{x}, score: %{y:.3f} +- %{customdata[0]:.3f}, sample size: %{customdata[1]} <extra>%{customdata[2]}</extra>"
)

    
    min_score = min(sorted_scores.min().min(), 0)

target                                                                      age  \
algorithm                                                               numbers   
fold                                                                    numbers   
metric                                                           n_participants   
main_category category                                                            
examination   Body__Measures                                              84160   
laboratory    Autoantibodies__-__Immunofluorescence__&__Immun...           4735   
              Cholesterol__-__Total                                       51199   
              Fatty__Acids__-__Serum                                       3152   
              Flame__Retardant__Metabolites__-__Urine__(Surplus)           4979   
              Hepatitis__E__:__IgG__&__IgM__Antibodies                    36480   
              Human__Papillomavirus__(HPV)__-__6,__11,__16__&...           6815   
              Lead__-__Dust                                                3227   
              Non-dioxin-like__Polychlorinated__Biphenyls                  3890   
              Personal__Care__and__Consumer__Product__Chemica...           5333   
              Pesticides__-__Current__Use__-__Urine__(Formerl...           4986   
questionnaire Creatine__Kinase                                            12343   
              Early__Childhood                                            33214   
              Hepatitis                                                   24816   
              Mental__Health__-__Panic__Disorder                            305   
              Osteoporosis                                                24015   
              Prostate__Conditions                                         5392   
              Prostate__Specific__Antigen__Follow-up                         81   
              Smoking__-__Cigarette_or_Tobacco__Use__-__Adult             15324   
              Taste__&__Smell                                              7418   
              Volatile__Toxicant__(Subsample)                              5925   

target                                                                        \
algorithm                                                                      
fold                                                                           
metric                                                           n_variables   
main_category category                                                         
examination   Body__Measures                                              19   
laboratory    Autoantibodies__-__Immunofluorescence__&__Immun...          36   
              Cholesterol__-__Total                                       15   
              Fatty__Acids__-__Serum                                      40   
              Flame__Retardant__Metabolites__-__Urine__(Surplus)          27   
              Hepatitis__E__:__IgG__&__IgM__Antibodies                    15   
              Human__Papillomavirus__(HPV)__-__6,__11,__16__&...          17   
              Lead__-__Dust                                               24   
              Non-dioxin-like__Polychlorinated__Biphenyls                 39   
              Personal__Care__and__Consumer__Product__Chemica...          19   
              Pesticides__-__Current__Use__-__Urine__(Formerl...          49   
questionnaire Creatine__Kinase                                            16   
              Early__Childhood                                            17   
              Hepatitis                                                   15   
              Mental__Health__-__Panic__Disorder                          36   
              Osteoporosis                                                19   
              Prostate__Conditions                                        17   
              Prostate__Specific__Antigen__Follow-up                      32   
 